<a href="https://colab.research.google.com/github/leess0980/Homework/blob/master/4.30%20pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip #wget을 통해 다음 경로의 파일을 가져옴
!unzip -qq cats_and_dogs_filtered.zip # unzip을 통해 압축을 해제, -qq를 통해 unzip의 terminal 출력을 하지않음

--2020-05-15 02:55:47--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.111.128, 2607:f8b0:4001:c03::80
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.111.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘cats_and_dogs_filtered.zip’

cats_and_dogs_filte 100%[===================>]  65.43M   141MB/s    in 0.5s    

2020-05-15 02:55:47 (141 MB/s) - ‘cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [7]:
from Training_Classification_From_Scratch import main

ImportError: ignored

In [5]:
main()

[1] Test Loss: 0.6726, Accuracy: 61.70%
model saved!
[2] Test Loss: 0.6614, Accuracy: 60.30%
[3] Test Loss: 0.6555, Accuracy: 61.60%
[4] Test Loss: 0.6508, Accuracy: 62.40%
model saved!
[5] Test Loss: 0.6481, Accuracy: 62.30%
[6] Test Loss: 0.6410, Accuracy: 63.40%
model saved!
[7] Test Loss: 0.6418, Accuracy: 63.90%
model saved!
[8] Test Loss: 0.6393, Accuracy: 63.50%
[9] Test Loss: 0.6341, Accuracy: 64.10%
model saved!
[10] Test Loss: 0.6330, Accuracy: 64.30%
model saved!
[11] Test Loss: 0.6331, Accuracy: 63.90%
[12] Test Loss: 0.6326, Accuracy: 64.30%
[13] Test Loss: 0.6299, Accuracy: 64.70%
model saved!
[14] Test Loss: 0.6270, Accuracy: 65.50%
model saved!
[15] Test Loss: 0.6323, Accuracy: 63.30%
[16] Test Loss: 0.6574, Accuracy: 60.40%
[17] Test Loss: 0.6257, Accuracy: 65.80%
model saved!
[18] Test Loss: 0.6254, Accuracy: 64.50%
[19] Test Loss: 0.6191, Accuracy: 66.70%
model saved!
[20] Test Loss: 0.6281, Accuracy: 64.60%
[21] Test Loss: 0.6200, Accuracy: 66.10%
[22] Test Loss: 0.

In [6]:
!nvidia-smi

Fri May 15 03:18:10 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P0    76W / 149W |    521MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
from Training_Classification_Transfer_Learning import main

In [10]:
main()

Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/checkpoints/mobilenet_v2-b0353104.pth



[1] Test Loss: 0.3537, Accuracy: 95.90%
model saved!
[2] Test Loss: 0.3668, Accuracy: 94.20%
[3] Test Loss: 0.3491, Accuracy: 96.20%
model saved!
[4] Test Loss: 0.3349, Accuracy: 97.90%
model saved!
[5] Test Loss: 0.3386, Accuracy: 97.30%


KeyboardInterrupt: ignored

In [0]:
# Inference_Classification
import os
import cv2
from PIL import Image
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import numpy as np
from Model_Scratch import MODEL as MODEL_SCRATCH
from Model_Transfer_Learning import MODEL as MODEL_TRANSFER

def load_label_map(textFile):
    return np.loadtxt(textFile, str, delimiter='\t')
    
def cv_image_read(image_path):
    print(image_path)
    return cv2.imread(image_path)

def print_result(inference_result, class_map):
    class_text = class_map[np.argmax(inference_result)]
    print(inference_result)
    print(class_text)

def inference_image(opencv_image, transform_info, model, DEVICE):
    image = Image.fromarray(opencv_image)
    image_tensor = transform_info(image)
    image_tensor = image_tensor.unsqueeze(0)
    image_tensor = image_tensor.to(DEVICE)
    result = model(image_tensor)
    return result


def inference_main(image_path, _is_transfer = False):
    USE_CUDA = torch.cuda.is_available()
    DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

    img_width, img_height = 224, 224
    transform_info = transforms.Compose([
                transforms.Resize(size=(img_width, img_height))
                , transforms.ToTensor()
                    ])
    class_map = load_label_map('label_map.txt')
    num_classes = len(class_map)
    
    model = None
    model_str = None
    if not _is_transfer:
        model = MODEL_SCRATCH(num_classes).to(DEVICE)
        model_str = "PyTorch_Classification_Model_Scratch"
    else:
        model = MODEL_TRANSFER(num_classes).to(DEVICE)
        model_str = "PyTorch_Classification_Model_Trnasfer"
    model_str += ".pt" 
    
    model.load_state_dict(torch.load(model_str))
    model.eval()

    opencv_image = cv_image_read(image_path)
    inference_result = inference_image(opencv_image, transform_info, model, DEVICE)
    inference_result = inference_result.cpu().detach().numpy()
    print_result(inference_result, class_map)

def main(index, _PATH = "/content/cats_and_dogs_filtered/validation"):
    PATH = _PATH
    validation_cats_dir = PATH + '/cats'  # directory with our validation cat pictures
    validation_dogs_dir = PATH + '/dogs'  # directory with our validation dog pictures
    list_of_test_cats_images = os.listdir(validation_cats_dir)
    list_of_test_dogs_images = os.listdir(validation_dogs_dir)
    for idx in range(len(list_of_test_cats_images)):
        list_of_test_cats_images[idx] = validation_cats_dir + '/'+list_of_test_cats_images[idx]
    for idx in range(len(list_of_test_dogs_images)):
        list_of_test_dogs_images[idx] = validation_dogs_dir + '/'+list_of_test_dogs_images[idx]
    list_of_test_images = list_of_test_cats_images + list_of_test_dogs_images
    inference_main(list_of_test_images[index])
    return list_of_test_images
    

In [13]:
list_of_images = main(600)

/content/cats_and_dogs_filtered/validation/dogs/dog.2317.jpg
[[0.39140248 0.6085975 ]]
dogs


In [14]:
inference_main(list_of_image[800], True)

NameError: ignored